In [94]:
from qutip import *
import numpy as np
π = np.pi

In [95]:
cutoff = 20

a = destroy(cutoff)
I2 = identity(2)
Ib = identity(cutoff)

def BS(theta,phi):
    term = tensor([a.dag(),identity(2),a,identity(2)])
    phase = np.exp(1j*phi)
    return (-1j*(theta/2)*(phase*term + np.conj(phase)*term.dag())).expm()

def D(alpha):
    return (alpha*a.dag() - np.conj(alpha)*a).expm()

def Dc(alpha, beta):
    return tensor(D(alpha), basis(2,0)*basis(2,0).dag()) + tensor(D(beta), basis(2,1)*basis(2,1).dag())
    
def CP():
    return (-1j*(np.pi/2)*tensor(a.dag()*a,sigmaz())).expm()

def DCX(alpha):
    return tensor(alpha*a.dag() - np.conj(alpha)*a, (I2 - sigmax())/2).expm()

def DCZ(alpha):
    return tensor(alpha*a.dag() - np.conj(alpha)*a, (I2 - sigmaz())/2).expm()
    

# Previous Gates

In [1]:
# conditional displacement operator 
def Dc(α, β, eigenbasis, qubit, subsystems):
    
    # define Hadamard gate
    H = Qobj([[1,1],[1,-1]],dims=[[2],[2]])/np.sqrt(2)
    
    # cavity_1 ⊗ qubit_1 + cavity_1 ⊗ qubit_1
    # returns D(α)⊗ |0><0| + D(β)⊗ |1><1|, D(α)⊗ |+><+| + D(β)⊗ |-><-|
    # or D(α)⊗ |+i><+i| + D(β)⊗ |-i><-i| depending on basis 
    
    # define X, Y, or Z basis projectors
    if eigenbasis == 'X':
        plus = H*basis(2, 0)
        minus = H*basis(2, 1)
        plus_projector = plus*plus.dag()
        minus_projector = minus*minus.dag()
        projector_α = plus_projector
        projector_β = minus_projector
    elif eigenbasis == 'Y':
        plus_i = (basis(2, 0) + 1j*basis(2, 1))/np.sqrt(2)
        minus_i = (basis(2, 0) - 1j*basis(2, 1))/np.sqrt(2)
        plus_i_projector = plus_i * plus_i.dag()
        minus_i_projector = minus_i * minus_i.dag()
        projector_α = plus_i_projector
        projector_β = minus_i_projector
    elif eigenbasis == 'Z':
        projector_α = basis(2,0)*basis(2,0).dag()
        projector_β = basis(2,1)*basis(2,1).dag()
    else:
        print('Please enter a valid eigenbasis')
    
    # return displacement operator
    if subsystems == 2: 
        return tensor(D(α), projector_α) + tensor(D(β), projector_β)
    elif subsystems == 4: 
        if qubit == 1:
            return tensor(D(α), projector_α, Ib, I2) + tensor(D(β), projector_β, Ib, I2)
        if qubit == 2:
            return tensor(Ib, I2, D(α), projector_α) + tensor(Ib, I2, D(β), projector_β)
    else: 
        print('Please enter an even number')

In [2]:
# controlled parity gate
# make it so that you specify mode # and target qubit #
def CP(pauli_operator, qubit, subsystems):
    """
    Returns control-parity gate that imparts σ_x, σ_y, or σ_z 
    to the qubit
    
    Args: 
        pauli_operator: 'X', 'Y', or 'Z' represents 
            σ_x, σ_y, or σ_z which will be applied
            to the qubit 
        subsystems: the total number of modes in qubits we consider 
                        
    Returns: 
        returns e^{-i π/2 a†a ⊗ σ_x (σ_y, σ_z)} depending on 
        value of Pauli operator 
    """
    
    # determine pauli_operator
    if pauli_operator == 'X':
        σ_i = σ_x
    elif pauli_operator == 'Y':
        σ_i = σ_y
    elif pauli_operator == 'Z':
        σ_i = σ_z
    else:
        print('Please enter valid Pauli operator, either \'X\', \'Y\', or \'Z\'')
        
    # returns e^{-i π/2 a†a ⊗ σ_i} where σ_i ∈ {σ_x, σ_y, σ_z}
    if subsystems == 2:
        return (-1j*(np.pi/2)*tensor(a.dag()*a, σ_i)).expm()
    elif subsystems == 4: 
        if qubit == 1:
            return (-1j*(np.pi/2)*tensor(a.dag()*a, σ_i, Ib, I2)).expm()
        elif qubit == 2: 
            return (-1j*(np.pi/2)*tensor(a.dag()*a, I2, Ib, σ_i)).expm()
        print("Please enter valid target qubit")
    else: 
        print('Please enter a valid number of subsystems ')
    
    # note that e^{-i π/2 a†a ⊗ σ_x} = tensor(Ib, H)* e^{-i π/2 a†a ⊗ σ_z}*tensor(Ib, H)
    # and e^{-i π/2 a†a ⊗ σ_y} = tensor(Ib, rx(-π/2))* e^{-i π/2 a†a ⊗ σ_z} *tensor(Ib, rx(π/2))

In [ ]:
display(CP('Z', 2, 4).dag()*Dc(-1j*α, 1j*α, 'Z', 1, 4)*CP('Z', 2, 4) == 
       (tensor(α*a.dag() - np.conj(α)*a, sigmaz(), Ib, sigmaz())).expm())
DZZ = lambda α: (tensor(α*a.dag() - np.conj(α)*a, sigmaz(), Ib, sigmaz())).expm()

In [ ]:
display(CP('X', 2, 4).dag()*Dc(-1j*α, 1j*α, 'X', 1, 4)*CP('X', 2, 4) == 
       (tensor(α*a.dag() - np.conj(α)*a, sigmax(), Ib, sigmax())).expm())
DXX = lambda α: (tensor(α*a.dag() - np.conj(α)*a, sigmax(), Ib, sigmax())).expm()

# $ R_{ZZ}(\theta) $

In [61]:
### ZZ gate

θ = np.random.rand()
α = np.sqrt(θ/4)

In [64]:
U1 = tensor(CD(-α,α),Ib,I2)
U2 = tensor(CD(1j*α,-1j*α),Ib,I2).permute([0,3,2,1])

U = U2.dag()*U1.dag()*U2*U1

P = tensor(basis(cutoff,0),I2,basis(cutoff,0),I2)
P.dag()*U*P == (-1j*θ*tensor(sigmaz(),sigmaz())/2).expm()

True

In [65]:
U1 = tensor(CD(-α,α),Ib,I2)
U2 = BS(π,π/2)*tensor(Ib,I2,CD(1j*α,-1j*α))*BS(π,-π/2)#.permute([0,3,2,1])

U = U2.dag()*U1.dag()*U2*U1

P = tensor(basis(cutoff,0),I2,basis(cutoff,0),I2)
P.dag()*U*P == (-1j*θ*tensor(sigmaz(),sigmaz())/2).expm()

True

In [ ]:
# DZ1(α, β) is a controlled displacement  that 
# applies D(α) if qubit 1 is |0> and D(β) if qubit is |1>
DZ1 = lambda α, β: Dc(α, β, 'Z', 1, 4)
DZ2 = lambda α, β: Dc(α, β, 'Z', 2, 4)

# # generates θ (random float between 0 and 1)
# and defines α as function of θ
θ = np.random.rand()
α = np.sqrt(θ/4)

# note that DZ1(-α,α) == (tensor(α*a.dag() - np.conj(α)*a, -σ_z, Ib, I2)).expm()
U1 = 

# note that DZ2(-α, α) == (tensor(Ib, I2, α*a.dag() - np.conj(α)*a, -σ_z)).expm()
# and DZ2(-α, α).permute([2,1,0,3]) == (tensor(α*a.dag() - np.conj(α)*a, I2, Ib, -σ_z)).expm()
U2 = DZ2(-α, α).permute([0,3,2,1])

# sequence of unitaries that carve out area in phase space 
U = U2.dag()*U1.dag()*U2*U1

# CNOT

In [113]:
### CNOT
α = np.sqrt(π/2)
U1 = tensor([Dc(0, α),I2])
U2 = tensor([DCX(1j*α),I2]).permute([0,2,1])

U = (U2.dag()*U1.dag()*U2*U1)

In [114]:
P = tensor(basis(cutoff,0).dag(),I2,I2)
P*U*P.dag()

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[1.0000000e+00+0.00000000e+00j 0.0000000e+00+0.00000000e+00j
  0.0000000e+00+0.00000000e+00j 0.0000000e+00+0.00000000e+00j]
 [0.0000000e+00+0.00000000e+00j 1.0000000e+00+0.00000000e+00j
  0.0000000e+00+0.00000000e+00j 0.0000000e+00+0.00000000e+00j]
 [0.0000000e+00+0.00000000e+00j 0.0000000e+00+0.00000000e+00j
  2.2845649e-11+8.34120823e-11j 1.0000000e+00-8.34120042e-11j]
 [0.0000000e+00+0.00000000e+00j 0.0000000e+00+0.00000000e+00j
  1.0000000e+00-8.34120370e-11j 2.2845553e-11+8.34120187e-11j]]

In [ ]:
# distance by which we displace oscillator in phase space
α = np.sqrt(π/2)

# DaZ(-α) is an asymmetric controlled displacement 
# and applies D(0) if qubit is |0> and D(α) if qubit is |1>
DaZ = lambda α: Dc(0, α, 'Z', 1, 2)

# DaX(-α) is an asymmetric controlled displacement 
# and applies D(0) if qubit is |+> and D(α) if qubit is |->
DaX = lambda α: Dc(0, α, 'X', 1, 2)

# asymmetric horizontal displacement (D(0) if qubit_1 is |o> and D(+α) if qubit_1 is |1>)
U1_cnot = tensor([DaZ(α), Ib, I2])

# this is the same as (tensor([DCX(1j*α), Ib, I2]).permute([0,3,2,1]))
U2_cnot = BS(π, π/2)*tensor(Ib, I2, DaX(1j*α))*BS(π, -π/2)

# manually define U2.dag() and U1.dag()
# U3 = tensor([DCZ(-α), Ib, I2])
# U4 = BS(π, π/2)*tensor(Ib, I2, DCX(-1j*α))*BS(π, -π/2)

# gate sequence that carves out area in phase space according to CNOT conditions
U_cnot = U2_cnot.dag()*U1_cnot.dag()*U2_cnot*U1_cnot
# U = U4 * U3 * U2 * U1

# projection operator that begins in the ground state
P = tensor(basis(cutoff,0),I2, basis(cutoff,0), I2)

# for CNOT gate sequence, trace out oscillator by assuming 
# we start and end in the ground state
CNOT_synthesized = P.dag()*U_cnot*P 
CNOT_synthesized

# TOFFOLI

In [ ]:
# distance by which we displace oscillator in phase space
α = np.sqrt(np.pi/2)

# define a controlled-parity opeartor that flips qubit_1 depending on parity of photons in cavity
# (flip if odd and no flip if even) conditioned upon 
cp = tensor(CP(),Ib,I2)

# asymmetric diplacement operators that applies D(0) (D(0)) if qubit_2 is in |0> and D(-iα/2) (D(α/2)) if qubit_2 is in |1>
d1 = tensor([Dc(0,-1j*α/2),Ib,I2]).permute([0,3,2,1])
d2 = tensor([Dc(0, α/2),Ib,I2]).permute([0,3,2,1])

# conditional-conditional displacement 
U12 = tensor([d2*cp*d1*cp.dag(),I2])

# apply D(0) if qubit_3 is in |+> and D(+iα) if qubit_3 is in |->
U3 = tensor([DCX(1j*α),I2,Ib,I2]).permute([0,3,2,1])

# gate sequence that carves out area in phase space according to CNOT conditions
Utoff = U3.dag()*U12.dag()*U3*U12

# trace out oscillator by assuming we start and end in the ground state
P = tensor(basis(cutoff,0),I2,basis(cutoff,0),I2)
P.dag()*Utoff*P 